# Further Pre-training MobileBERT MLM with Federated Averaging (Shakepeare)

In [1]:
# Copyright 2020, The TensorFlow Federated Authors.
# Copyright 2020, Ronald Seoh
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Google Colab settings

In [2]:
# Use Google Colab
use_colab = True

# Is this notebook running on Colab?
# If so, then google.colab package (github.com/googlecolab/colabtools)
# should be available in this environment

# Previous version used importlib, but we could do the same thing with
# just attempting to import google.colab
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    # Mount Google Drive root directory
    drive.mount('/content/drive')

    # cd to the appropriate working directory under my Google Drive
    %cd '/content/drive/My Drive/Colab Notebooks/BERTerated'
    
    # List the directory contents
    !ls

In [3]:
# IPython reloading magic
%load_ext autoreload
%autoreload 2

In [4]:
# Install required packages
!pip install -r requirements.txt

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


## Import packages

In [5]:
import os
import sys
import random
import datetime

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_text as tf_text
import transformers

import nest_asyncio
nest_asyncio.apply()

import fedavg
import fedavg_client
import datasets
import utils

# Random seed settings
random_seed = 692
random.seed(random_seed) # Python
np.random.seed(random_seed) # NumPy
tf.random.set_seed(random_seed) # TensorFlow

# Tensorflow GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Test the TFF is working:
tff.federated_computation(lambda: 'Hello, World!')()

Num GPUs Available:  1


b'Hello, World!'

In [6]:
# Print version information
print("Python version: " + sys.version)
print("NumPy version: " + np.__version__)
print("TensorFlow version: " + tf.__version__)
print("TensorFlow Federated version: " + tff.__version__)
print("Transformers version: " + transformers.__version__)

Python version: 3.6.9 (default, Jul 17 2020, 12:50:27) 
[GCC 8.4.0]
NumPy version: 1.18.5
TensorFlow version: 2.3.1
TensorFlow Federated version: 0.17.0
Transformers version: 3.4.0


In [7]:
!nvidia-smi

Sat Nov 21 19:42:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 455.23.05    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  On   | 00000000:04:00.0 Off |                  N/A |
|  0%   30C    P2    56W / 250W |  10525MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

## Experiment Settings

In [9]:
TOTAL_ROUNDS = 3 # Number of total training rounds
ROUNDS_PER_EVAL = 1 # How often to evaluate

TRAIN_CLIENTS_PER_ROUND = 3 # How many clients to sample per round.
CLIENT_EPOCHS_PER_ROUND = 6 # Number of epochs in the client to take per round.

BATCH_SIZE = 8 # Batch size used on the client.
TEST_BATCH_SIZE = 16 # Minibatch size of test data.

# Maximum length of input token sequence for BERT.
BERT_MAX_SEQ_LENGTH = 32 # Some small number, just to see if our codes are working

# Optimizer configuration
SERVER_LEARNING_RATE = 1.0 # Server learning rate.
CLIENT_LEARNING_RATE = 2e-6 # Client learning rate

# Path to save trained weights and logs
RESULTS_DIRECTORY = os.path.join(
    '.', 'results',
    'mobilebert_mlm_shakespeare_fedavg',
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
)

RESULTS_LOG = os.path.join(RESULTS_DIRECTORY, "logs")
RESULTS_MODEL = os.path.join(RESULTS_DIRECTORY, "model")

## Dataset

In [10]:
train_client_data, test_client_data = tff.simulation.datasets.shakespeare.load_data(cache_dir='./tff_cache')

### Tokenizer

In [11]:
mobilebert_tokenizer = transformers.MobileBertTokenizer.from_pretrained(
    'google/mobilebert-uncased', cache_dir='./transformers_cache')

In [12]:
# Imitate transformers tokenizer with TF.Text Tokenizer
tokenizer_tf_text, vocab_lookup_table, special_ids_mask_table = datasets.preprocessing_for_bert.convert_huggingface_tokenizer(mobilebert_tokenizer)

In [13]:
# Test if our new tokenizer works
ttt = tokenizer_tf_text.tokenize("This is a test.")
print("TF Text tokenizer output shape:", tf.shape(ttt.to_tensor()))
print(tf.squeeze(ttt.to_tensor(), axis=-1))
mobilebert_tokenizer.decode(tf.squeeze(ttt, axis=-1).to_list()[0])

TF Text tokenizer output shape: tf.Tensor([1 5 1], shape=(3,), dtype=int32)
tf.Tensor([[2023 2003 1037 3231 1012]], shape=(1, 5), dtype=int32)


'this is a test.'

### Preprocessing

In [14]:
def check_empty_snippet(x):
    return tf.strings.length(x['snippets']) > 0

def tokenizer_and_mask_wrapped(x):

    masked, labels = datasets.preprocessing_for_bert.tokenize_and_mask(tf.reshape(x['snippets'], shape=[1]),
                                                                       max_seq_length=BERT_MAX_SEQ_LENGTH,
                                                                       bert_tokenizer_tf_text=tokenizer_tf_text,
                                                                       vocab_lookup_table=vocab_lookup_table,
                                                                       special_ids_mask_table=special_ids_mask_table,
                                                                       cls_token_id=mobilebert_tokenizer.cls_token_id,
                                                                       sep_token_id=mobilebert_tokenizer.sep_token_id,
                                                                       pad_token_id=mobilebert_tokenizer.pad_token_id,
                                                                       mask_token_id=mobilebert_tokenizer.mask_token_id)

    return (masked, labels)

def preprocess_for_train(train_dataset):
    return (
        train_dataset
        # Tokenize each samples using MobileBERT tokenizer
        .map(tokenizer_and_mask_wrapped, num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)
        # Shuffle
        .shuffle(100000)
        # Form minibatches
        # Use drop_remainder=True to force the batch size to be exactly BATCH_SIZE
        # and make the shape **exactly** (BATCH_SIZE, SEQ_LENGTH)
        .batch(BATCH_SIZE, drop_remainder=True)
        # Repeat to make each client train multiple epochs
        .repeat(count=CLIENT_EPOCHS_PER_ROUND)
    )
    
def preprocess_for_test(test_dataset):
    return (
        test_dataset
        # Tokenize each samples using MobileBERT tokenizer
        .map(tokenizer_and_mask_wrapped, num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)
        # Shuffle
        .shuffle(100000)
        # Form minibatches
        # Use drop_remainder=True to force the batch size to be exactly TEST_BATCH_SIZE
        # and make the shape **exactly** (TEST_BATCH_SIZE, SEQ_LENGTH)
        .batch(TEST_BATCH_SIZE, drop_remainder=True)
    )   

### Training set

In [15]:
train_client_data = train_client_data.preprocess(preprocess_fn=lambda x: x.filter(check_empty_snippet))

In [16]:
train_client_data = train_client_data.preprocess(preprocess_fn=preprocess_for_train)

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [17]:
print(train_client_data.element_type_structure)

(TensorSpec(shape=(8, 32), dtype=tf.int32, name=None), TensorSpec(shape=(8, 32), dtype=tf.int32, name=None))


### Test set

In [18]:
test_client_data_all_merged = test_client_data.create_tf_dataset_for_client(
    test_client_data.client_ids[0]).filter(check_empty_snippet)

if len(test_client_data.client_ids) > 1:
    for i in range(1, len(test_client_data.client_ids)):
        test_client_data_all_merged = test_client_data_all_merged.concatenate(
            test_client_data.create_tf_dataset_for_client(test_client_data.client_ids[i]).filter(check_empty_snippet))

In [19]:
test_client_data_all_merged = preprocess_for_test(test_client_data_all_merged)

In [20]:
print(test_client_data_all_merged.element_spec)

(TensorSpec(shape=(16, 32), dtype=tf.int32, name=None), TensorSpec(shape=(16, 32), dtype=tf.int32, name=None))


## Model

In [21]:
mobilebert_model = transformers.TFMobileBertForPreTraining.from_pretrained(
    'google/mobilebert-uncased', cache_dir='./transformers_cache')

All model checkpoint layers were used when initializing TFMobileBertForPreTraining.

All the layers of TFMobileBertForPreTraining were initialized from the model checkpoint at google/mobilebert-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMobileBertForPreTraining for predictions without further training.


In [22]:
# Due to the limitations with Keras subclasses, we can only use the main layer part from pretrained models
# and add output heads by ourselves
mobilebert_keras_converted = utils.convert_huggingface_mlm_to_keras(
    huggingface_model=mobilebert_model,
    max_seq_length=BERT_MAX_SEQ_LENGTH,
)

In [23]:
# Use lists of NumPy arrays to backup pretained weights
mobilebert_pretrained_trainable_weights = []
mobilebert_pretrained_non_trainable_weights = []

for w in mobilebert_keras_converted.trainable_weights:
    mobilebert_pretrained_trainable_weights.append(w.numpy())

for w in mobilebert_keras_converted.non_trainable_weights:
    mobilebert_pretrained_non_trainable_weights.append(w.numpy())

In [24]:
def tff_model_fn():
    """Constructs a fully initialized model for use in federated averaging."""

    loss = utils.MaskedLMCrossEntropy()

    model_wrapped = utils.KerasModelWrapper(
        tf.keras.models.clone_model(mobilebert_keras_converted),
        train_client_data.element_type_structure, loss)

    return model_wrapped

## Training

In [25]:
%tensorboard --logdir {RESULTS_LOG}

### Training setups

In [26]:
summary_writer = tf.summary.create_file_writer(RESULTS_LOG)

In [27]:
def server_optimizer_fn():
    return tf.keras.optimizers.SGD(learning_rate=SERVER_LEARNING_RATE)

def client_optimizer_fn():
    return tf.keras.optimizers.SGD(learning_rate=CLIENT_LEARNING_RATE)

In [28]:
iterative_process = fedavg.build_federated_averaging_process(
    model_fn=tff_model_fn,
    model_input_spec=train_client_data.element_type_structure,
    initial_trainable_weights=mobilebert_pretrained_trainable_weights,
    initial_non_trainable_weights=mobilebert_pretrained_non_trainable_weights,
    server_optimizer_fn=server_optimizer_fn, 
    client_optimizer_fn=client_optimizer_fn)

Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


In [29]:
server_state = iterative_process.initialize()

In [30]:
metric_eval = utils.MaskedLMCrossEntropyMetric(name='test_accuracy')

In [31]:
model_final = tff_model_fn() # The one to store the final weights

### Training loop

In [32]:
with summary_writer.as_default():
    for round_num in range(1, TOTAL_ROUNDS + 1):

        # Training clients selection
        print("Choosing clients to use for training...")

        sampled_clients = np.random.choice(
            train_client_data.client_ids,
            size=TRAIN_CLIENTS_PER_ROUND,
            replace=False)

        sampled_train_data = [
            train_client_data.create_tf_dataset_for_client(client)
            for client in sampled_clients
        ]

        print("Training clients selection complete.")

        # FedAvg
        print(f'Round {round_num} start!')

        server_state, train_loss = iterative_process.next(server_state, sampled_train_data)

        print(f'Round {round_num} training loss: {train_loss}')
        
        # Write down train_metrics to the log
        tf.summary.scalar('train_loss', train_loss, step=round_num)

        # Evaluation
        if round_num % ROUNDS_PER_EVAL == 0:
            model_final.from_weights(server_state.model_weights)

            # Test dataset generation for this round
            print("Calculating validation metric:")

            validation_metric = utils.keras_evaluate(
                model_final.keras_model, test_client_data_all_merged, metric_eval)

            print(f'Round {round_num} validation metric: {validation_metric}')
            
            # Write down train_metrics to the log
            tf.summary.scalar('validation_metric', validation_metric, step=round_num)

Choosing clients to use for training...
Training clients selection complete.
Round 1 start!
Client 19769 : updated the model with server message.
Client 19256 : updated the model with server message.
Client 8510 : updated the model with server message.
Client 19769 : training start.
Client 19256 : training start.
Client 8510 : training start.
Client 19256 : training finished. 0  examples processed, loss: 0
Client 8510 : training finished. 0  examples processed, loss: 0
Client 19769 : 1  batch, 8  examples processed
Client 19769 : 2  batch, 16  examples processed
Client 19769 : 3  batch, 24  examples processed
Client 19769 : 4  batch, 32  examples processed
Client 19769 : 5  batch, 40  examples processed
Client 19769 : 6  batch, 48  examples processed
Client 19769 : 7  batch, 56  examples processed
Client 19769 : 8  batch, 64  examples processed
Client 19769 : 9  batch, 72  examples processed
Client 19769 : 10  batch, 80  examples processed
Client 19769 : 11  batch, 88  examples process

### Save the trained model

In [33]:
model_final.keras_model.save(RESULTS_MODEL)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: ./results/mobilebert_mlm_shakespeare_fedavg/20201121-194207/model/assets


INFO:tensorflow:Assets written to: ./results/mobilebert_mlm_shakespeare_fedavg/20201121-194207/model/assets
